In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Height"
cohort = "GSE71994"

# Input paths
in_trait_dir = "../../input/GEO/Height"
in_cohort_dir = "../../input/GEO/Height/GSE71994"

# Output paths
out_data_file = "../../output/preprocess/Height/GSE71994.csv"
out_gene_data_file = "../../output/preprocess/Height/gene_data/GSE71994.csv"
out_clinical_data_file = "../../output/preprocess/Height/clinical_data/GSE71994.csv"
json_path = "../../output/preprocess/Height/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains PBMC gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Height data is available in row 4
trait_row = 4
# Age data is available in row 3
age_row = 3
# Gender data is available in row 1
gender_row = 1

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert height data to continuous values."""
    try:
        # Extract the value after colon and convert to float
        if isinstance(value, str) and ':' in value:
            height_str = value.split(':', 1)[1].strip()
            return float(height_str)
        return None
    except:
        return None

def convert_age(value):
    """Convert age data to continuous values."""
    try:
        # Extract the value after colon and convert to integer
        if isinstance(value, str) and ':' in value:
            age_str = value.split(':', 1)[1].strip()
            return int(age_str)
        return None
    except:
        return None

def convert_gender(value):
    """Convert gender data to binary values: female=0, male=1."""
    try:
        if isinstance(value, str) and ':' in value:
            gender_str = value.split(':', 1)[1].strip().lower()
            if 'female' in gender_str:
                return 0
            elif 'male' in gender_str:
                return 1
        return None
    except:
        return None

# 3. Save Metadata
# Perform initial filtering on dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait_row is not None, extract clinical features
if trait_row is not None:
    # Get the clinical data from the previous step
    # (assuming clinical_data is available from the previous step)
    try:
        # Use geo_select_clinical_features to extract clinical data
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data, 
            trait=trait, 
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(clinical_features)
        print(f"Clinical features preview: {preview}")
        
        # Save the clinical features to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file, index=True)
        print(f"Clinical features saved to {out_clinical_data_file}")
    except NameError:
        print("clinical_data not available from previous step")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 4: Gene Identifier Review

In [ ]:
# Examining the gene identifiers in the gene data
# The IDs appear to be numeric identifiers (e.g., 7896746) which are not standard
# human gene symbols. Human gene symbols are typically alphanumeric (like BRCA1, TP53, etc.)
# These appear to be probe IDs from a microarray platform that need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Analyze the gene identifiers in gene expression data and gene annotation

# Based on examining the gene expression data and gene annotation data:
# - The gene expression data uses numeric identifiers (e.g., 7896746) in the 'ID' column
# - The gene annotation data has these same identifiers in the 'ID' column
# - The gene symbols can be extracted from the 'gene_assignment' column

# 2. Create a mapping between probe IDs and gene symbols
# The 'gene_assignment' column has a complex format with gene symbols embedded
# We need to extract gene symbols from this column

# Define the mapping columns
probe_col = 'ID'  # This column matches the index in gene_data
gene_col = 'gene_assignment'  # This column contains the gene symbols

# Create a mapping dataframe
mapping_df = pd.DataFrame({
    'ID': gene_annotation['ID'].astype(str),
    'Gene': gene_annotation[gene_col]
})

# Preview the mapping before processing
print("Original mapping preview (first 2 rows):")
print(mapping_df.head(2))

# 3. Apply the gene mapping to transform probe-level data to gene-level data
# The apply_gene_mapping function will:
# - Extract human gene symbols from the gene_assignment text
# - Handle many-to-many mappings with proper distribution of expression values
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)

# Preview the gene expression data
print("\nTransformed gene expression data (first 5 genes, first 3 samples):")
print(gene_data.iloc[:5, :3])

# Check the shape of the transformed data
print(f"\nGene expression data shape: {gene_data.shape}")

# Save the gene data to the specified path
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Let's use the gene data from Step 6 that already has genes mapped from probes
# We need to normalize these gene symbols
try:
    # Normalize gene symbols using the NCBI Gene database information
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Check if we have any genes after normalization
    if normalized_gene_data.shape[0] == 0:
        print("WARNING: No genes remained after normalization. This may indicate an issue with gene symbol mapping.")
        is_gene_available = False
    else:
        is_gene_available = True
        
    # Save the normalized gene data to the output file (even if empty, for logging purposes)
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error during gene normalization: {e}")
    normalized_gene_data = pd.DataFrame()
    is_gene_available = False

# 2. Load clinical data from the processed file
try:
    clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
    print(f"Loaded clinical data with shape: {clinical_df.shape}")
    print(f"Clinical data columns: {clinical_df.columns.tolist()}")
    
    # Check if trait column exists in the data
    if trait not in clinical_df.columns:
        clinical_df[trait] = np.nan  # Add empty trait column
        print(f"Added empty '{trait}' column to clinical data")
    
    is_trait_available = not clinical_df[trait].isna().all()
    print(f"Trait availability: {is_trait_available}")
    
except FileNotFoundError:
    print("Clinical data file not found. Creating a new clinical dataframe.")
    clinical_df = pd.DataFrame(index=gene_data.columns)
    clinical_df[trait] = np.nan  # Empty trait column
    clinical_df['Age'] = np.nan  # Empty age column
    clinical_df['Gender'] = np.nan  # Empty gender column
    is_trait_available = False

# 3. Create linked data
linked_data = pd.DataFrame(index=clinical_df.index)
linked_data[trait] = clinical_df[trait]

# Add demographic columns if available
if 'Age' in clinical_df.columns:
    linked_data['Age'] = clinical_df['Age']
if 'Gender' in clinical_df.columns:
    linked_data['Gender'] = clinical_df['Gender']

# Add gene expression data if available
if is_gene_available:
    for gene in normalized_gene_data.index:
        linked_data[gene] = normalized_gene_data.loc[gene]

print(f"Linked data shape: {linked_data.shape}")

# Handle missing values only if trait data is available
if is_trait_available:
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # Check if trait and demographic features are biased
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
else:
    is_biased = False  # Not biased because there's no trait data to be biased

# Prepare notes based on data availability
if not is_gene_available and not is_trait_available:
    note = "Dataset lacks both gene expression data (after normalization) and Height measurements."
elif not is_gene_available:
    note = "Dataset lacks gene expression data after normalization (all gene symbols may be non-standard)."
elif not is_trait_available:
    note = "Dataset contains gene expression data but no Height measurements."
else:
    note = "Dataset contains both gene expression and Height measurements."

# 5. Final validation
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if it's usable
print(f"Dataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset cannot be used for association studies due to missing or biased data.")


### Step 8: Data Normalization and Linking

In [ ]:
import numpy as np
import os

# 1. Let's load the gene expression data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
gene_data = get_genetic_data(matrix_file)

# Normalize gene symbols using the NCBI Gene database information
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link clinical and genetic data
# In Step 2, we determined that Height data is not available in this dataset (trait_row = None)
# Create a minimal clinical dataframe with the trait column (containing NaNs)
sample_ids = normalized_gene_data.columns
minimal_clinical_df = pd.DataFrame(index=sample_ids)
minimal_clinical_df[trait] = np.nan  # Add the trait column with NaN values

# If we have age and gender data from Step 2, add those columns
if age_row is not None:
    minimal_clinical_df['Age'] = get_feature_data(clinical_data, age_row, 'Age', convert_age).iloc[0]

if gender_row is not None:
    minimal_clinical_df['Gender'] = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender).iloc[0]

minimal_clinical_df.index.name = 'Sample'

# Save this minimal clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
minimal_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Create a linked dataset 
linked_data = pd.concat([minimal_clinical_df, normalized_gene_data.T], axis=1)
linked_data.index.name = 'Sample'
print(f"Linked data shape: {linked_data.shape}")

# We would normally handle missing values here, but since all trait values are missing,
# the dataset will be empty after removing samples with missing trait values
# Therefore, we'll skip that step

# 4 & 5. Validate and save cohort information
# Since trait_row was None in Step 2, we know Height data is not available
is_gene_available = len(normalized_gene_data) > 0
is_trait_available = False  # Height data is not available

note = "Dataset contains gene expression data but no Height measurements. This dataset is not usable for studying Height associations."

# For datasets without trait data, we set is_biased to False
# This indicates the dataset is not usable due to missing trait data, not due to bias
is_biased = False

# Final validation
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Since there is no trait data, the dataset is not usable for our association study
# So we should not save it to out_data_file
print(f"Dataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset does not contain Height data and cannot be used for association studies.")